# Converting jsons file from the sketchy website

In [ ]:
import pandas as pd
import json

In [ ]:
import pandas as pd

weather_data = pd.DataFrame(columns=["date"])
for month in range(1,12+1):
    with open(f'data/jsons_weather_monthly/{str(month).zfill(2)}.json') as f:
        data = json.load(f)

    for daily in data["data"]["weather"]:
        hourly_data = []
        for hourly in daily["hourly"]:
            hourly_data.append(pd.DataFrame.from_dict(hourly))
        daily_data = pd.concat(hourly_data)
        daily_data["date"] = daily["date"]
        weather_data = pd.concat([weather_data, daily_data])

weather_data.to_csv("data/weather_hourly.csv")

## Combine the datasets in one csv file

### filter interesting columns

In [ ]:
weather_data = weather_data[['date', 'time', 'tempC','windspeedKmph','weatherCode','precipMM','humidity','pressure','cloudcover','WindChillC','WindGustKmph','FeelsLikeC','uvIndex']]

In [ ]:
import csv
import datetime
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Read weather data
weather_data = {}

with open("data/weather_hourly.csv", "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        date = row["date"]
        time = row["time"]
        hour = int(time) // 100
        dt = datetime.datetime.strptime(date, "%Y-%m-%d").replace(hour=hour)
        weather_data[dt] = row
"""
# Read and encode occupation data
occupation_df = pd.read_csv("data/ocupacio_enginyeria_2022.csv")
columns_to_encode = ['Espai', 'Estudi', 'Activitat', 'Modalitat docencia']

#for col in columns_to_encode:
#    label_encoder = LabelEncoder()
#    occupation_df[col] = label_encoder.fit_transform(occupation_df[col].astype(str))

occupation_data = {}

for _, row in occupation_df.iterrows():
    if not row["Data inicial"].strip():  # Check if the date string is empty
        continue
    start_date = datetime.datetime.strptime(row["Data inicial"], "%d/%m/%Y")
    start_hour = int(row["Hora inicial"].split(":")[0])
    start_dt = start_date.replace(hour=start_hour)
    occupation_data[start_dt] = row.to_dict()

"""
# Read target data
target_data = {}

with open("data/Consum horari electricitat Enginyeries 2022.csv", "r") as f:
    reader = csv.DictReader(f, delimiter=";")
    for row in reader:
        date = datetime.datetime.strptime(row["Date"], "%d/%m/%Y")
        hour = int(row["Hour"].split(":")[0])
        dt = date.replace(hour=hour)
        target_data[dt] = row

# Merge the data
merged_data = []
for dt in sorted(weather_data.keys()):
    merged_row = weather_data[dt]
    """
    if dt in occupation_data:
        merged_row.update(occupation_data[dt])
    """
    if dt in target_data:
        merged_row.update(target_data[dt])

    merged_data.append(merged_row)

# Save the merged data to a CSV file
with open("data/merged_data.csv", "w", newline="") as f:
    fieldnames = set()
    for row in merged_data:
        fieldnames.update(row.keys())
    fieldnames = list(fieldnames)

    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for row in merged_data:
        writer.writerow(row)


In [ ]:
merged_df = pd.read_csv("data/merged_data.csv")

In [ ]:

merged_df = pd.read_csv("data/merged_data.csv")

#columns_to_drop = ['Unnamed: 0', 'Data inicial', 'Data final', 'Hora inicial', 'Hora final', 'time', 'weatherIconUrl', 'winddir16Point', 'Date', 'Hour', 'Observacions', 'tempF', 'WindChillF', 'HeatIndexF', 'FeelsLikeF', 'DewPointF', 'WindGustMiles', 'visibilityMiles', 'pressureInches', 'precipInches']
columns_to_drop = ['Unnamed: 0', 'weatherIconUrl', 'winddir16Point', 'Date', 'Hour', 'tempF', 'WindChillF', 'weatherDesc','HeatIndexF', 'FeelsLikeF', 'DewPointF', 'WindGustMiles', 'visibilityMiles', 'pressureInches', 'precipInches']
merged_df = merged_df.drop(columns=columns_to_drop)
merged_df["time"] = merged_df["time"]/100
# Save the updated DataFrame to a new CSV file
coloms_with_coma_decimal = ["Q-Enginyeria (Cos Central) [kWh] [Q-Enginyeria]","Q-Enginyeria (Espina 4) [kWh] [Q-Enginyeria]","Q-Enginyeria (Química) [kWh] [Q-Enginyeria]"]
for col in coloms_with_coma_decimal:
    changed = []
    for i,val in enumerate(merged_df[col]):
        if val == val:
            changed.append(val.replace(",","."))
        else: #we got a nan. interpolate by having previous value
            changed.append(changed[-1])
    
    merged_df[col] = changed

merged_df.to_csv("data/updated_merged_data.csv", index=False)


In [ ]:
merged_df = pd.read_csv("data/updated_merged_data.csv")

In [ ]:
merged_df.columns

Note: In the future we could play with [observations, Estudi,Modalitat docencia] columns and NLP to get some representation of the activity being done and its impact

In [ ]:
occupation_data = pd.read_csv("data/ocupacio_enginyeria_2022.csv")

# Creating and embedding for the title of activities

In [ ]:
import torch

The problem we have is that we have too many activities to create a one hot vector of each of them.
Second problem we have is that if we wanted to add a new activity this would break the model

In [ ]:
#All activities we have.
occupation_data

In the future it would be cool to create our own model that creates some embedding optimized for owr problem.

For now we'll use some pretrained.

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
start_token = "[CLS] "
end_token = " [SEP]"
test_txt = start_token + "This class is chaotic and boring at the same time. " + end_token
tokenized_text = tokenizer.tokenize(test_txt)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
segments_ids = [1] * len(tokenized_text)

In [ ]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-multilingual-cased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()
model.to("mps")

In [ ]:
# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens]).to("mps")
segments_tensors = torch.tensor([segments_ids]).to("mps")

In [ ]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

In [ ]:
# `token_vecs` is a tensor with sha pe [Ntokens x 768]
token_vecs = hidden_states[-2][0] #second to last hiden layer

# Calculate the average of all Ntokens token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)
sentence_embedding.shape

In [ ]:
def sentence2hidden_states(txt):
    test_txt = start_token + txt + end_token
    tokenized_text = tokenizer.tokenize(test_txt)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text) 
    segments_ids = [1] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens]).to("mps")
    segments_tensors = torch.tensor([segments_ids]).to("mps")
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
    
    hidden_states = outputs[2]
    return hidden_states

Here we have an embedding for each sentence of size (768)

Now we'll try to apply kmeans to kluster all titles by their semantic meaning.

In [ ]:
from tqdm import tqdm 

In [ ]:
vectors = {}
h_states = {}
occupations = set()
for occu in tqdm(occupation_data["Activitat"]):
    if occu not in occupations:
        h = sentence2hidden_states(occu)[-2][0].to("cpu") #get all layers first batch
        h_states[occu] =h

        # Calculate the average of all Ntokens token vectors.
        sentence_embedding = torch.mean(h, dim=0)
        vectors[occu] = sentence_embedding.numpy()
        occupations.add(occu) 

We'll do the same but encode the classroom the activity is in the text also

In [ ]:
vectors_plus = {}
h_states_plus = {}
occupations_plus = set()
for occu, espai in zip(occupation_data["Activitat"],occupation_data["Espai"]):
    if occu + " " + espai not in occupations_plus:
        h = sentence2hidden_states(occu + " " + espai)[-2][0].to("cpu") #get all layers first batch
        h_states_plus[occu + " " + espai] = h
        
        # Calculate the average of all Ntokens token vectors.
        sentence_embedding = torch.mean(h, dim=0)
        vectors_plus[occu + " " + espai] = sentence_embedding.numpy()
        occupations_plus.add(occu + " " + espai) 

Save data to pickle file for later use

In [ ]:
import pickle

#bert_data = {
#    "single_ocu":{"mean_vect":vectors,"h_states":h_states},
#    "ocu_plus_space":{"mean_vect":vectors_plus,"h_states":h_states_plus}
#    }

bert_data = {
    "single_ocu":{"mean_vect":vectors},
    "ocu_plus_space":{"mean_vect":vectors_plus}
    }

# Open a file and use dump()
with open('data/bert_embedded.pkl', 'wb') as file:
      
    pickle.dump(bert_data, file)

We'll do some fast clustering to see if it makes any sense

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, random_state=0, n_init="auto").fit(list(vectors_plus.values()))
cluster_cent = kmeans.cluster_centers_

In [ ]:
cluster_map = pd.DataFrame()
cluster_map['data_index'] = list(vectors_plus.keys())
cluster_map['cluster'] = kmeans.labels_

In [ ]:
cluster_map[cluster_map["cluster"]==0].value_counts()

In [ ]:
cluster_map[cluster_map["cluster"]==1].value_counts()

In [ ]:
cluster_map[cluster_map["cluster"]==2].value_counts()

In [ ]:
cluster_map[cluster_map["cluster"]==3].value_counts()

We'll not sure if it makes any sense XD

We'll add to the dataset a column for the hiddent states of bert. We'll pass a text through bert if the input class doesn't exist.

# Creating dataset class

In [ ]:
from torch.utils.data import Dataset, DataLoader
from sklearn.decomposition import PCA
from sklearn import preprocessing 
import torch
import numpy as np
import pandas as pd
import pickle
import datetime

In [ ]:
class energyProject_dataset(Dataset):
    def __init__(self,dataset_pth,occupacio_pth,bert_embeddings_pkl_pth,pca_pkl_pth=None):
        self.df = pd.read_csv(dataset_pth)
        with (open(bert_embeddings_pkl_pth, "rb")) as openfile:
            self.bert_embeddings = pickle.load(openfile)
        self.activitivity_encoding_mode = 0
        
        #order of values in target tensor will follow this
        self.target_labels = ["Q-Enginyeria (Cos Central) [kWh] [Q-Enginyeria]","Q-Enginyeria (Espina 4) [kWh] [Q-Enginyeria]","Q-Enginyeria (Química) [kWh] [Q-Enginyeria]"]
       
        #load occupation data
        self.occupation_df = pd.read_csv(occupacio_pth)
        #we'll remove entries without date
        self.occupation_df = self.occupation_df[self.occupation_df["Data inicial"] != " "]

        #convert date string to be in the form y-m-d instead of d/m/y
        #convert hour data to datetime object so we can compare them
        for i, row in self.occupation_df.iterrows(): 
            self.occupation_df.loc[i]["Data inicial"] =  datetime.datetime.strptime(self.occupation_df.loc[i]["Data inicial"], "%d/%m/%Y").strftime("%Y-%m-%d")
            self.occupation_df.loc[i]["Hora inicial"] = datetime.datetime.strptime(self.occupation_df.loc[i]["Hora inicial"] ,"%H:%M").time()
            self.occupation_df.loc[i]["Hora final"] = datetime.datetime.strptime(self.occupation_df.loc[i]["Hora final"] ,"%H:%M").time()
        
        self.ocup_vocab = list(set(self.occupation_df["Activitat"]))
        self.espais_vocab = list(set(self.occupation_df["Espai"]))
        #Add a padding occupation
        self.espais_vocab.append("NO ESPAI")

        #Normalize all climate data to be between 0-1
        #we'll keep their scaler objects so we can transform their values back to original
        #and not loose meaning.
        self.column_scalers = {}
        columns_to_process = ['winddirDegree', 'precipMM', 'visibility', 'WindChillC',
       'humidity', 'pressure','windspeedMiles', 'uvIndex', 'DewPointC',
       'FeelsLikeC', 'tempC','weatherCode','HeatIndexC', 'WindGustKmph', 'cloudcover',
       'windspeedKmph','Q-Enginyeria (Cos Central) [kWh] [Q-Enginyeria]',
       'Q-Enginyeria (Química) [kWh] [Q-Enginyeria]',
       'Q-Enginyeria (Espina 4) [kWh] [Q-Enginyeria]'] #normalze also target
     
        for col in columns_to_process:
            scaler, values = self.normalize_values(self.df[col])
            self.df[col] = values
            self.column_scalers[col] = scaler
        
        #If we want this dataset to work with batches in modes different than 0 and 1.
        #We need to know what is the maximum number of activities at the same time so we can
        #padd the samples smaller. We need the batch to have the same shape samples every time.

        #we'll use that we are iterating throught this to compute all the one_hot vectors of the encodings
        #and fit a PCA object so we can have it with dimensionality reduction.

        self.max_ocu_lenght = 0
        self.all_one_hots = []
        for i, row in self.df.iterrows():
            day2day_ocu = self.activty_class_perT(row["date"],row["time"])
            
            #find largest occupation size per hour
            day2day_ocu_l = len(day2day_ocu)
            if day2day_ocu_l > self.max_ocu_lenght:
                self.max_ocu_lenght = day2day_ocu_l
                
            if pca_pkl_pth == True: #if we have to calculate the PCA
                #compute one_hot vectors of occupation at this time
                self.all_one_hots.append(self.activity_class_one_hot(day2day_ocu))
        
        if (pca_pkl_pth == True): #nan value, recalculate and save file
            self.all_one_hots = np.array(self.all_one_hots)
            #compute PCA on all the one_hot vectors
            self.ocu_one_hot_pca = PCA(n_components=1000) #1000 size output vector (number chosen by hand)
            self.ocu_one_hot_pca.fit(self.all_one_hots)
            # Open a file and use dump()
            with open('data/pca_occupation.pkl', 'wb') as file:
                pickle.dump(self.ocu_one_hot_pca, file)
        else:
            with (open(pca_pkl_pth, "rb")) as openfile:
                self.ocu_one_hot_pca = pickle.load(openfile)
        
        ##free memory by removing unnecessary variables.
        del self.all_one_hots

    def normalize_values(self,x):
        """
        Input a list of values
        Output a sklearn scaler object and the list normalized.
        We need to keep the scaler to be able to re-scale the data back and now what value it is in reality.
        """
        to_scale = np.array(x).reshape(-1, 1) #the library needs this extra dimensions trick to interpret properly
        min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
        x_scaled = min_max_scaler.fit_transform(to_scale)
        return min_max_scaler, x_scaled
    
    def denormalize_values(self,x_n,scaler):
        """
        Given some set of values and a sklearn scaler object
        Transform back the values to their original "space"ArithmeticError
        return: set of values same shape as input
        """
        to_scale = np.array(x_n).reshape(-1, 1)
        return scaler.inverse_transform(to_scale)

    def datetime_enc(self,date, time)->torch.tensor:
        """
        Encodes incoming date and time strings as two values for each that 
        come from infering the index value on a sin function and cos function.
        Its nice beacause we encode the smoothness and circularity of the trigonometric
        functions.

        input <- (date: str, time:str)
        output -> (torch.tensor((1,5)))
        """
        
        date_obj =  datetime.datetime.strptime(date, "%Y-%m-%d")
        
        #encode year as floas
        year_enc = float(date_obj.year)/1000 # divide by 100 to have reasonable value
        
        ##
        ##Encoding: (sin, cos) value for each day month
        ##
        idx_d = date_obj.timetuple().tm_yday #day of the year number
        date_enc= [np.sin((idx_d/365) * 2*np.pi),np.cos((idx_d/365) * 2*np.pi)]  #365 days a yar +1 offset so we don't have negative value
        
        #encode time by hour in the day
        time_enc = [np.sin((time/24) * 2*np.pi),np.cos((time/24) * 2*np.pi)]  #24 hours a day. +1 offset so we don't have negative values
        
        return torch.tensor([year_enc]+date_enc+time_enc) 

    def datetime_dec(self,enc_tens):
        """
        Decodes incoming encoded date and time tensor as the two respective
        date time string values

        input <- (torch.tensor([torch.float,torch.float]))
        output -> date: str, time:str
        """
        #decode year
        year = int(enc_tens[0].item() * 1000)

        #decode date
        penc_date = np.arctan2(enc_tens[1].item(),enc_tens[2].item()) / (2*np.pi) * 365
        date = datetime.datetime(year, 1, 1) + datetime.timedelta(penc_date - 1)
        date = date.strftime("%Y-%m-%d")

        #decode time
        time = np.round((np.arctan2(enc_tens[3].item(),enc_tens[4].item()) / (2*np.pi) * 24)% 24)
        
        return date,time

    def activty_class_perT(self,date,time)->pd.DataFrame:
        """
        Returns slice of the pd Dataframe of activities active given some date and time
        """
        #filter dataset to see activities that day
        day2day_ocu = self.occupation_df[self.occupation_df["Data inicial"] == date ] 
        h = datetime.time(int(time))
        hour2hour_ocu = day2day_ocu[(day2day_ocu["Hora inicial"] <= h) & (day2day_ocu["Hora final"] > h)]
        return pd.DataFrame(hour2hour_ocu)

    def activity_class_one_hot(self,activities)->np.array:
        """ 
        returns flattenned coocurrence one-hot matrix of activities and classrooms
        """
        occurrence_matrix = np.zeros((len(self.ocup_vocab),len(self.espais_vocab)))
        for i,actv in activities.iterrows(): #iterate found activities
            ocup_idx = self.ocup_vocab.index(actv["Activitat"])
            espais_idx = self.espais_vocab.index(actv["Espai"])
            occurrence_matrix[ocup_idx,espais_idx] = 1
            
        #now flatten the occurrence matrix into a one hot vector
        one_hot = occurrence_matrix.flatten()
        return one_hot
    
    def class_one_hot(self,activitats)->torch.tensor:
        """
        Returns "one hot" encoding of activitat.
        In reality will not be a true one hot but a list of indexes
        that can later on be passed to some embedding layer
        """
        activitats["Espai"]
        one_hot_esp = [self.espais_vocab.index(key) for key in activitats["Espai"]]
        return torch.tensor(one_hot_esp)
       


    def __len__(self):
        return len(self.df) 
    
    def __getitem__(self, index):
        """
        This function will return more than one object depending on the mode it is on.
        Activity encoding mode:
            0: I.very large one-hot encoding of all the combinations of classroom and activity concatenated
                with all other features.
               II. target values
            
            1:  I. 0 but first with some PCA applied to reduce dimensionality of 
                the enormous one-hot encoding.
                II. target values
            
            2: returns four objects, 
                I.mean_encoding of activity from bert(as many as activities at time stamp), 
                II. classroom one hot encoding for each activity(as many as activities at time stamp)
                III. All other features at that time stamp
                IV. target values
            
            2.5:  returns four objects
                I. embedding tensor of activity from bert(as many as activities at time stamp), 
                II. classroom one hot encoding for each activity(as many as activities at time stamp)
                III. All other features at that time stamp
                IV. target values
            
            3: returns three objects,
                I. mean encoding of activity, classroom pair through bert (as many as activities at time stamp),
                II. All other features at that time stamp
                III. target values
            
            3.5:  returns four objects
                I. embedding tensor of activity, classroom pair through bert(as many as activities at time stamp), 
                II. classroom one hot encoding for each activity(as many as activities at time stamp)
                III. All other features at that time stamp
                IV. target values
            
            4: returns three objects, raw data thought for model handling.
                I. activities in text form paired with their classroom
                II. All other features at that time stamp
                III. target values
        """

        #get row in df for data to be evaluated:
        row = self.df.iloc[index]

        #First getting the "all other data " features tensor
        #Date-time encoding tensor
        enc_dt_tens = self.datetime_enc(row["date"],row["time"]) #date-time encoding

        #weather data tensor
        weather_tens = torch.tensor(row.drop(["date","time","Q-Enginyeria (Cos Central) [kWh] [Q-Enginyeria]","Q-Enginyeria (Espina 4) [kWh] [Q-Enginyeria]","Q-Enginyeria (Química) [kWh] [Q-Enginyeria]"]))

        #second get the target values tensor
        target_tens = torch.tensor(row[self.target_labels])
        
        #Get activities at given time and date
        activities = self.activty_class_perT(row["date"],row["time"])
        if self.activitivity_encoding_mode <= 1:
            one_hot = self.activity_class_one_hot(activities)
            
            if(self.activitivity_encoding_mode == 0):
                one_hot = torch.tensor(one_hot)
                #return values
                sample = {'x': torch.cat((enc_dt_tens,weather_tens,one_hot),axis=0), 'y': target_tens}
                return sample
            
            if(self.activitivity_encoding_mode == 1):
                one_hot = one_hot.reshape(1, -1) #create extra dimension because this counts as only one sample
                smaller_x = torch.tensor(self.ocu_one_hot_pca.transform(one_hot))[0] #get rid of extra dim
                sample = {'x': torch.cat((enc_dt_tens,weather_tens,smaller_x),axis=0), 'y': target_tens}
                return sample
        
        if self.activitivity_encoding_mode == 2:
            #get bert embeddings
            emb_activ = []
            for i,actv in activities.iterrows(): #iterate found activities   
                emb_activ.append(self.bert_embeddings["ocu_plus_space"]["mean_vect"][actv["Activitat"] + " " + actv["Espai"]])
            emb_activ = torch.tensor(np.array(emb_activ))

            #get espai one-hot
            espai_one_hot = self.class_one_hot(activities)

            #padd with occupation 0 vector so all samples are same shape and espai with ""NO ESPAI"
            if emb_activ.shape[0] < self.max_ocu_lenght: 
                emb_activ = torch.cat((emb_activ,torch.zeros((self.max_ocu_lenght-emb_activ.shape[0],768))),axis=0)
                espai_padd = pd.DataFrame(columns=["Espai"]) 
                espai_padd["Espai"] = ["NO ESPAI"]* (self.max_ocu_lenght-len(espai_one_hot))
                padd_one_hot = self.class_one_hot(espai_padd)
                espai_one_hot = torch.cat((espai_one_hot,padd_one_hot))
                
            sample = {'ocu_ber_emb': emb_activ,'espai_enc':espai_one_hot, "general_data":torch.cat((enc_dt_tens,weather_tens),axis=0), 'y': target_tens}
            return sample
        
        if self.activitivity_encoding_mode == 2.5:
            assert "NOT IMPLEMENTED YET"
            pass
            
        if self.activitivity_encoding_mode == 3:
            emb_activ = []
            for i,actv in activities.iterrows(): #iterate found activities   
                emb_activ.append(self.bert_embeddings["ocu_plus_space"]["mean_vect"][actv["Activitat"] + " " + actv["Espai"]])
            emb_activ = torch.tensor(np.array(emb_activ))

            if emb_activ.shape[0] < self.max_ocu_lenght: #padd with occupation 0 vector so all samples are same shape
                emb_activ = torch.cat((emb_activ,torch.zeros((self.max_ocu_lenght-emb_activ.shape[0],768))),axis=0)
            
            #return values
            sample = {'activ': torch.tensor(emb_acti), 'general_data':torch.cat((enc_dt_tens,weather_tens),axis=0), 'y': target_tens}
            return sample
        
        if self.activitivity_encoding_mode == 3.5:
            assert "NOT IMPLEMENTED YET"
            #errors to solve have to recalculate bert passing
            emb_acti = []
            for i,actv in activities.iterrows(): #iterate found activities   
                emb_acti.append(self.bert_embeddings["ocu_plus_space"]["h_states"][actv["Activitat"] + " " + actv["Espai"]])
                print(self.bert_embeddings["ocu_plus_space"]["h_states"][actv["Activitat"] + " " + actv["Espai"]].shape)
            emb_acti = torch.tensor(emb_acti)
            
            print(emb_acti.shape)
            #return values
            return torch.tensor(emb_acti), torch.cat((enc_dt_tens,weather_tens),axis=0) , target_tens

        if self.activitivity_encoding_mode == 4:
            assert "NOT IMPLEMENTED YET"
            pass

In [ ]:
dataset = energyProject_dataset("data/updated_merged_data.csv","data/ocupacio_enginyeria_2022.csv","data/bert_embedded.pkl",True)

Save dataset as pickle so we don't have to run init every time and it is only one file to handle

In [ ]:
# Open a file and use dump()
with open('data/dataset_class.pkl', 'wb') as file:
    pickle.dump(dataset, file)

# Usage instructions:

In [ ]:
#import dataset class
from dataset import energyProject_dataset
from torch.utils.data import DataLoader
import pickle
import torch
import torch.nn as nn
import numpy as np
import tqdm
import torch
import torch.nn as nn
import pandas as pd
import yaml

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import wandb

!wandb login

In [ ]:
#load dataset object file
with (open('data/dataset_class.pkl', "rb")) as openfile:
    dataset = pickle.load(openfile)

In [ ]:
#now you can create a dataloader and use it!
dataloader = DataLoader(dataset, batch_size=100, shuffle=False)

In [ ]:
next(iter(dataloader))

By default the dataset is in 0 mode. Here are the modes:

In [ ]:
dataset.__getitem__?

To change the dataset mode just do:

In [ ]:
dataset.activitivity_encoding_mode = 2 #or any value

And you don't even have to update the dataloader!

In [ ]:
next(iter(dataloader))

In [ ]:
# Get one batch of data
batch = next(iter(dataloader))

# Print the keys of the batch dictionary
print("Batch keys:", batch.keys())

# Print the shape of each item in the batch
for key in batch.keys():
    print(f"Shape of batch['{key}']:", batch[key].shape)

# Print the first few examples in the batch
print("First few examples in the batch:")
for key in batch.keys():
    print(f"batch['{key}'][0]:", batch[key][0])

## GRU/LSTM approach

In [ ]:
class GRUModel(nn.Module):
    """
    A PyTorch GRU model for regression tasks, which takes three inputs and predicts a continuous output.

    Args:
        input_size (int): The number of expected features in the input 'ocu_ber_emb' and 'espai_enc'
        hidden_size (int): The number of features in the hidden state of the GRU.
        output_size (int): The number of expected features in the output.
        num_layers (int): Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two GRUs together to form a `stacked GRU`.
        general_data_size (int): The number of features in the 'general_data' input.

    Inputs: ocu_ber_emb, espai_enc, general_data
        - **ocu_ber_emb** of shape `(batch, seq_len, input_size - 1)`: tensor containing the features of the 'ocu_ber_emb' input sequence.
          The input can also be a packed variable length sequence.
        - **espai_enc** of shape `(batch, seq_len)`: tensor containing the features of the 'espai_enc' input sequence.
        - **general_data** of shape `(batch, general_data_size)`: tensor containing the features of the 'general_data' input.

    Outputs: output
        - **output** of shape `(batch, output_size)`: tensor containing the output features from the last layer of the GRU, for each t.
    """
    def __init__(self, input_size, hidden_size, output_size, num_layers, general_data_size):
        super(GRUModel, self).__init__()
        
        # GRU layer for 'ocu_ber_emb' and 'espai_enc'
        self.gru = nn.GRU(input_size + 1, hidden_size, num_layers, batch_first=True)
        
        # Linear layer to reduce 'general_data' dimension
        self.linear1 = nn.Linear(general_data_size, hidden_size)
        
        # Final linear layer for output
        self.linear2 = nn.Linear(2 * hidden_size, output_size)
        
    def forward(self, ocu_ber_emb, espai_enc, general_data):
        # Expand 'espai_enc' dimensions to concatenate with 'ocu_ber_emb'
        espai_enc = espai_enc.unsqueeze(2)
        
        # Concatenate 'ocu_ber_emb' and 'espai_enc'
        x = torch.cat((ocu_ber_emb, espai_enc), dim=2)
        
        # GRU forward pass
        out, _ = self.gru(x)
        
        # Use only the final output of the GRU
        out = out[:, -1, :]
        
        # Pass 'general_data' through linear layer
        general_data_out = self.linear1(general_data)
        
        # Concatenate GRU output and 'general_data' output
        out = torch.cat((out, general_data_out), dim=1)
        
        # Final linear layer
        out = self.linear2(out)
        
        return out


In [ ]:
#useful stuff to get the model parameters

# Initialize empty lists to hold ocu_codes and espai_codes
ocu_codes = []
espai_codes = []

# Loop over the dataset
for i in range(len(dataset)):
    sample = dataset[i]
    ocu_codes.append(sample['ocu_ber_emb'])
    espai_codes.append(sample['espai_enc'])

# Convert lists to pandas Series to easily count unique values
ocu_codes = pd.Series(ocu_codes)
espai_codes = pd.Series(espai_codes)

n_ocu_codes = ocu_codes.nunique()
n_espai_codes = espai_codes.nunique()

print(f"Number of unique ocu_codes: {n_ocu_codes}")
print(f"Number of unique espai_codes: {n_espai_codes}")



In [ ]:
general_data_sizes = []

# Loop over the dataset
for i in range(len(dataset)):
    sample = dataset[i]
    general_data_sizes.append(len(sample['general_data']))

# Convert list to pandas Series to easily get max value
general_data_sizes = pd.Series(general_data_sizes)

general_data_vector_size = general_data_sizes.max()

print(f"General data vector size: {general_data_vector_size}")


In [ ]:
ocu_ber_emb = sample['ocu_ber_emb']
print(ocu_ber_emb.shape)

In [ ]:
input_size = 768  # Size of 'ocu_ber_emb' 
hidden_size = 256  # Hyperparameter, can be adjusted
num_layers = 2  # Hyperparameter, can be adjusted
output_size = 3  # Size of 'y'
general_data_size = 21  # Size of 'general_data'

gru_model = GRUModel(input_size, hidden_size, output_size, num_layers, general_data_size)


In [ ]:
## this is the working version of the training loop, without the wandb.

# # Instantiate the loss function and optimizer
# criterion = nn.MSELoss()  # Change this if your task is not regression
# optimizer = torch.optim.Adam(model.parameters())

# # Number of epochs
# num_epochs = 100

# # Path to save model
# model_save_path = 'gru_model.pt'

# # Training loop
# best_loss = float('inf')
# for epoch in range(num_epochs):
#     gru_model.train()  # Set the model to training mode
#     epoch_losses = []
#     progress_bar = tqdm.tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
#     for batch in progress_bar:
#         # Move tensors to the correct device
#         ocu_ber_emb = batch['ocu_ber_emb'].to(device).float()
#         espai_enc = batch['espai_enc'].to(device).float()
#         general_data = batch['general_data'].to(device).float()
#         y = batch['y'].to(device).float()

#         # Zero the gradients
#         optimizer.zero_grad()

#         # Forward pass
#         output = model(ocu_ber_emb, espai_enc, general_data)
#         # Compute loss
#         loss = criterion(output, y)

#         # Perform backward pass
#         loss.backward()

#         # Perform optimization
#         optimizer.step()

#         # Record the loss
#         epoch_losses.append(loss.item())

#         # Update progress bar
#         progress_bar.set_postfix({'loss': sum(epoch_losses) / len(epoch_losses)})

#     # Compute the average loss for this epoch
#     avg_loss = sum(epoch_losses) / len(epoch_losses)
#     print(f"Epoch {epoch+1}/{num_epochs} Loss: {avg_loss}")

#     # Save the model if it has the best loss yet
#     if avg_loss < best_loss:
#         best_loss = avg_loss
#         torch.save(gru_model.state_dict(), model_save_path)
#         print(f"Model saved at {model_save_path}")

In [ ]:
def nested_dict(original_dict):
    nested_dict = {}
    for key, value in original_dict.items():
        parts = key.split(".")
        d = nested_dict
        for part in parts[:-1]:
            if part not in d:
                d[part] = {}
            d = d[part]
        d[parts[-1]] = value
    return nested_dict


In [ ]:
from tqdm import tqdm
from typing import Dict

def train(config: Dict = None):
    # Weights & Biases Configuration
    with wandb.init(config=config):
        # unpack config dictionaries
        config = wandb.config
        config = nested_dict(config)
        
        optimizer_config = config["optimizer"] #TODO find out why it needs to be like this and config['optimizer']['type'] doesn't work
        
        # Fixed model parameters
        input_size = 768
        output_size = 3
        general_data_size = 21

        # Instantiate the model using the configuration parameters
        gru_model = GRUModel(input_size=input_size, 
                            hidden_size=config['hidden_size'], 
                            num_layers=config['num_layers'], 
                            output_size=output_size, 
                            general_data_size=general_data_size)
        gru_model = gru_model.to(device)

        # Instantiate the loss function
        criterion = nn.MSELoss()  # Change this if your task is not regression

        # Define the optimizer based on the configuration
        if optimizer_config["type"] == 'adam':
            optimizer = torch.optim.Adam(gru_model.parameters(), lr = 0.01)
        elif optimizer_config["type"] == 'sgd':
            optimizer = torch.optim.SGD(gru_model.parameters(), lr = 0.01)

        
        # Number of epochs
        num_epochs = 10

        # Training loop
        best_loss = float('inf')
        for epoch in range(num_epochs):
            gru_model.train()  # Set the model to training mode
            epoch_losses = []
            progress_bar = tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}")
            for batch in progress_bar:
                # Move tensors to the correct device
                ocu_ber_emb = batch['ocu_ber_emb'].to(device)
                espai_enc = batch['espai_enc'].to(device)
                general_data = batch['general_data'].to(device)
                y = batch['y'].to(device)

                # Zero the gradients
                optimizer.zero_grad()

                # Forward pass
                output = gru_model(ocu_ber_emb.float(), espai_enc.float(), general_data.float())
                # Compute loss
                loss = criterion(output, y.float())

                # Perform backward pass
                loss.backward()

                # Perform optimization
                optimizer.step()

                # Record the loss
                epoch_losses.append(loss.item())
                progress_bar.set_postfix({'Batch Loss': loss.item()})

            # Compute the average loss for this epoch
            avg_loss = sum(epoch_losses) / len(epoch_losses)
            wandb.log({'Epoch_Loss': avg_loss})  # Log the average loss for this epoch to Weights & Biases

            # Save the model if it has the best loss yet
            if avg_loss < best_loss:
                best_loss = avg_loss
                torch.save(gru_model.state_dict(), 'gru_model.pt')
                wandb.save('gru_model.pt')
                print(f"Model saved at {'gru_model.pt'}")

        # Close the Weights & Biases run
        wandb.finish()


In [ ]:
with open('config.yaml', 'r') as stream:
    try:
        sweep_config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

sweep_id = wandb.sweep(sweep_config, project="energy_project_uab")
wandb.agent(sweep_id, function=train)


# ML approach

# Any other approach

# Considerations about using Weights and Biases for tracking the experiments

#### 1. Install the Weights & Biases (wandb) library

If you haven't installed the wandb library yet, you can do so using pip. Run the following command in a new code cell:

```!pip install wandb```

#### 2. Import wandb into your script

Add the following line at the beginning of your script to import the wandb library:


```import wandb```

#### 3. Login to your W&B account

You need to login to your W&B account before you can start logging data. Run the following command in a new code cell:

```!wandb login```

You'll be asked for your API key, which you can get from the W&B website.

#### 4. Initialize wandb in your script

Now you can initialize wandb in your script. Replace "my-project" and "run-name" with your desired project and run names:

```wandb.init(project="my-project", name="run-name")```

#### 5. Log your model's hyperparameters

You can log any of your model's hyperparameters using the wandb.config object:

```
config = wandb.config
config.learning_rate = 0.01
config.batch_size = 32
config.epochs = 10
... any other hyperparameters
```

#### 6. Log metrics like loss and accuracy

To log metrics such as loss and accuracy, use the wandb.log method inside your training loop:

```for epoch in range(config.epochs):
    # training code here ...
    wandb.log({"loss": loss, "accuracy": accuracy})
```

#### 7. Save your model weights with wandb

```
torch.save(model.state_dict(), "model.pth")
wandb.save("model.pth")
```
